# Analysis Notebook - Count Genes and Events

This notebook processes the raw counts as provided by rMATS and performs some descriptive statistical analysis. It is used to produce the following outputs. 

## Data files created by this notebook
Output text files are written to the ``data/`` directory (at the same level as the ``jupyter`` directory). 

1. **gene_AS.tsv**: Alternative splicing events per gene
2. **genesWithCommonAS.tsv
3. **Total_AS_by_chr.tsv**: Total alternative splicing events per chromosome
4. **Total_AS_by_geneSymbol.tsv**: Count the number of tissues in which specific genes show significant alternative splicing
5. **Total_AS_by_tissue.tsv**: Count the number of significant splicing events per tissue
6. **Total_AS_by_splicingtype.tsv**: Count number of significant splicing events for each of the 5' alternative splicing categories
7. **Significant_AS_events.tsv**: ?? Counts of significant events per slicing type per tissue
8. **SplicingIndex_chr.tsv**: Splicing index by chr (number of sigificant AS events per 1000 exons)

In [2]:
defaultW <- getOption("warn")  # suppress warnings for this cell
options(warn = -1) 

library(dplyr)
library(ggplot2)
library(limma)
library(multtest)
library(Biobase)
library(edgeR)
library(tibble)
library(R.utils)
library(rtracklayer)

options(warn = defaultW)

## 1. Download all the rMATS results

Each of the alternative splicing output files are downloaded here:

### 1.1 get released rMATS GTF annotations

For each splicing type, the junctions are defined, so we have 5 specific annotated splicing specific junction ID annotation files:

1. **fromGTF.A3SS.txt**: annotations for the alternative 3' splice site junctions
2. **fromGTF.A5SS.txt**: annotations for the alternative 5' splice site junctions
3. **fromGTF.MXE.txt**: annotations for the mutually exclusive exon junctions
4. **fromGTF.RI.txt**: annotations for the retained introns junctions
5. **fromGTF.SE.txt**: annotations for the skipped exon junctions

## 1.2 Unpack the data.tar file if necessary
To run this script, we need to import three compressed files and unpack them

1. DGE_splicing_data.tar.gz
2. data.tar.gz
3. DGE_gene_csv.2020-06-17.tar.gz

In [53]:
dge_splicing_file_dir <- list.files("../../mounted-data", pattern='DGE_splicing_data.tar.gz')
dge_splicing_file <- paste("../../mounted-data", dge_splicing_file_dir, 'robinson-bucket/notebooks/DGE_splicing_data', sep='/')
dge_splicing_file_tar_gz <- paste(dge_splicing_file, '.tar.gz', sep='')
message("In order to unpack the necessary files, execute the following commands on the shell.")
message("1. DGE_splicing_data.tar.gz")
mycommand = paste("tar xvfz ",dge_splicing_file_tar_gz, "-C ../data", sep=" ")
message(mycommand)
message("2. data.tar.gz")
data_file_dir <- list.files("../../mounted-data", pattern='-data.tar.gz')
data_file_tar_gz <- paste("../../mounted-data", data_file_dir, 'robinson-bucket/notebooks/data.tar.gz', sep='/')
mycommand = paste("tar xvfz ",data_file_tar_gz, "-C ../data", sep=" ")
message(mycommand)
message("3. DGE_gene_csv.2020-06-17.tar.gz")
dge_file <- list.files("../../mounted-data", pattern='DGE_gene_csv')
dge_file_tar_gz <- paste("../../mounted-data", dge_file, 'robinson-bucket/results-download/DGE_gene_csv.2020-06-17.tar.gz', sep='/')
mycommand = paste("tar xvfz ",dge_file_tar_gz, "-C ../data", sep=" ")
message(mycommand)

In order to unpack the necessary files, execute the following commands on the shell.

1. DGE_splicing_data.tar.gz

tar xvfz  ../../mounted-data/5ee8da36143fa00113e810be-DGE_splicing_data.tar.gz-5ee8da36143fa00113e810be/robinson-bucket/notebooks/DGE_splicing_data.tar.gz -C ../data

2. data.tar.gz

tar xvfz  ../../mounted-data/5ee8da28143fa00113e8105a-data.tar.gz-5ee8da28143fa00113e8105a/robinson-bucket/notebooks/data.tar.gz -C ../data

3. DGE_gene_csv.2020-06-17.tar.gz

tar xvfz  ../../mounted-data/5ee9f327143fa00113eeb1fe-DGE_gene_csv.2020-06-17.tar.gz-5ee9f327143fa00113eeb1fe/robinson-bucket/results-download/DGE_gene_csv.2020-06-17.tar.gz -C ../data



In [16]:
## get the rmats 3.2.5 discovered/annotated junction information in GTF format
message("Decompressing fromGTF.tar.gz into ../data")
system("mkdir -p ../data && tar xvfz ../data/fromGTF.tar.gz -C ../data", intern = TRUE)
system("gunzip ../data/fromGTF.*txt.gz", intern = TRUE)
message("Done!\n")

Decompressing fromGTF.tar.gz into ../data



[1] "fromGTF.A3SS.txt.gz"             "fromGTF.A5SS.txt.gz"            
 [3] "fromGTF.MXE.txt.gz"              "fromGTF.RI.txt.gz"              
 [5] "fromGTF.SE.txt.gz"               "fromGTF.novelEvents.A3SS.txt.gz"
 [7] "fromGTF.novelEvents.A5SS.txt.gz" "fromGTF.novelEvents.MXE.txt.gz" 
 [9] "fromGTF.novelEvents.RI.txt.gz"   "fromGTF.novelEvents.SE.txt.gz"

Warning message in system("gunzip ../data/fromGTF.*txt.gz", intern = TRUE):
“running command 'gunzip ../data/fromGTF.*txt.gz' had status 2”


character(0)
attr(,"status")
[1] 2
attr(,"errmsg")
[1] "Resource temporarily unavailable"

Done!




### 2  Refined results
We define **refined results* as (FC > 1.5 and pVal < 0.05) for the sex\*as_event coefficient result for the linear model

### 2.1 getTissueReduction

In [28]:
tissue_reduction_filename <- "../assets/tissues.tsv"
tissue_reduction <- read.table(tissue_reduction_filename, header=TRUE, sep="\t",
                               skipNul=FALSE, stringsAsFactors = FALSE)
colnames(tissue_reduction)  <- c("SMTSD","female","male","include","display_name")
tissue_reduction <- tissue_reduction[tissue_reduction$display_name != "n/a",]
tissue_reduction$display_name <- factor(tissue_reduction$display_name)
levels(tissue_reduction$display_name)
message("We extracted ", length(levels(tissue_reduction$display_name))," different tissues with at least 50 samples in both M & f")

[1] "Adipose (sc)"          "Adipose (v)"           "Adrenal gland"        
 [4] "Aorta"                 "Atrial appendage"      "Breast"               
 [7] "Caudate"               "Cerebellar hemisphere" "Cerebellum"           
[10] "Coronary artery"       "Cortex"                "EBV-lymphocytes"      
[13] "Esophagus (gej)"       "Esophagus (m)"         "Esophagus (mu)"       
[16] "Fibroblasts"           "Frontal cortex"        "Hippocampus"          
[19] "Hypothalamus"          "Left ventricle"        "Liver"                
[22] "Lung"                  "Nucleus accumbens"     "Pancreas"             
[25] "Pituitary"             "Putamen"               "Sigmoid colon"        
[28] "Skeletal muscle"       "Skin (exposed)"        "Skin (not exposed)"   
[31] "Small intestine"       "Spinal cord"           "Spleen"               
[34] "Stomach"               "Thyroid"               "Tibial artery"        
[37] "Tibial nerve"          "Transverse colon"      "Whole blood"

We extracted 39 different tissues with at least 50 samples in both M & f



### 2.2 Read in refined results and annotations

In [29]:
significant_results_dir = "../data/"
pattern = "DGE_sex_as_events_refined.csv"
files <- list.files(path = significant_results_dir, pattern = pattern)
as_types <- c("a3ss", "a5ss", "mxe", "ri", "se")
length(files)

[1] 190

In [30]:
a3ss_annot <- read.table(file = "../data/fromGTF.A3SS.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
a5ss_annot <- read.table(file = "../data/fromGTF.A5SS.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
mxe_annot <- read.table(file = "../data/fromGTF.MXE.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
ri_annot <- read.table(file = "../data/fromGTF.RI.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)
se_annot <- read.table(file = "../data/fromGTF.SE.txt", sep = "\t", quote = "\"", header = T, stringsAsFactors = F)

In [31]:
head(se_annot)

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,ENSG00000034152.18,MAP2K3,chr17,+,21287990,21288091,21284709,21284969,21295674,21295769
2,2,ENSG00000034152.18,MAP2K3,chr17,+,21303182,21303234,21302142,21302259,21304425,21304553
3,3,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21287990,21288091,21296085,21296143
4,4,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21287990,21288091,21298412,21298479
5,5,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21284710,21284969,21296085,21296143
6,6,ENSG00000034152.18,MAP2K3,chr17,+,21295674,21295769,21284710,21284969,21298412,21298479


In [32]:
gene_as = data.frame()
GeneJunction <- rep("NA", length(files))
counts <- rep(NA, length(files))
ASE <- rep("NA", length(files))
Tissue <- rep("NA", length(files))
Display <- rep("NA", length(files))
GeneSymbol <- rep("NA", length(files))
GeneID <- rep("NA", length(files))
chr <- rep("NA", length(files))

for (i in 1:length(files)) {
    lines <- read.csv(paste0(significant_results_dir, files[i]),header = TRUE)
    event     <- as.vector(as.character(rownames(lines)))
    tissue1 <- gsub("_DGE_sex_as_events_refined.csv","", files[i], fixed = TRUE)
    counts[i] <- dim(lines)[1]
    if (counts[i] > 0) {
        event_idx <- substring(event, regexpr("[0-9]+$", event))
        event_idx
        if (grepl("^a3ss_", files[i])) {
            # remove the first 5 letters of the string 
            tissue2 <- substring(tissue1,6)
            ASE[i] <- "A3SS"
            Tissue[i] <- tissue2
            idx <- match(event_idx, a3ss_annot$ID)
            res <- data.frame(GeneJunction = event,
                              ASE = "A3SS", 
                              ASE_IDX = idx,
                              Tissue = tissue2,
                              counts = counts[i],
                              Display = tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol = a3ss_annot$geneSymbol[idx],
                              GeneID     = a3ss_annot$GeneID[idx],
                              chr = a3ss_annot$chr[idx])
            gene_as <- rbind(gene_as,res)

        } else if (grepl("^a5ss_", files[i])) {
            # remove the first 5 letters of the string 
            tissue2 <- substring(tissue1,6)
            ASE[i] <- "A5SS"
            Tissue[i] <- tissue2
            idx <- match(event_idx, a5ss_annot$ID)
            res <- data.frame(GeneJunction = event,
                              ASE = "A5SS", 
                              ASE_IDX = idx,
                              Tissue = tissue2,
                              counts = counts[i],
                              Display = tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol = a5ss_annot$geneSymbol[idx],
                              GeneID     = a5ss_annot$GeneID[idx],
                              chr = a5ss_annot$chr[idx])
            gene_as <- rbind(gene_as,res)
        } else if (grepl("^mxe_", files[i])) {
            ASE[i] <- "MXE"
            # remove the first 4 letters of the string 
            tissue2 <- substring(tissue1,5)
            Tissue[i] <- tissue2
            idx <- match(event_idx, a3ss_annot$ID)
            res <- data.frame(GeneJunction = event,
                              ASE = "MXE", 
                              ASE_IDX = idx,
                              Tissue = tissue2, 
                              counts = counts[i],
                              Display = tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol = mxe_annot$geneSymbol[idx],
                              GeneID     = mxe_annot$GeneID[idx],
                              chr = mxe_annot$chr[idx])
            gene_as <- rbind(gene_as,res)
        } else if (grepl("^se_", files[i])) {
            ASE[i] <- "SE"
            # remove the first 3 letters of the string 
            tissue2 <- substring(tissue1,4)
            Tissue[i] <- tissue2
            idx <- match(event_idx, se_annot$ID)
            res <- data.frame(GeneJunction = event,
                              ASE = "SE", 
                              ASE_IDX = idx,
                              Tissue = tissue2,
                              counts = counts[i],
                              Display = tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol = se_annot$geneSymbol[idx],
                              GeneID     = se_annot$GeneID[idx],
                              chr = se_annot$chr[idx])
            gene_as <- rbind(gene_as,res)
        } else if (grepl("^ri_", files[i])){
            ASE[i] <- "RI"
            # remove the first 3 letters of the string 
            tissue2 <- substring(tissue1,4)
            Tissue[i] <- tissue2
            idx <- match(event_idx, ri_annot$ID)
            res <- data.frame(GeneJunction = event,
                              ASE = "RI", 
                              ASE_IDX = idx,
                              Tissue = tissue2,
                              counts = counts[i],
                              Display = tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"],
                              GeneSymbol = ri_annot$geneSymbol[idx],
                              GeneID     = ri_annot$GeneID[idx],
                              chr = ri_annot$chr[idx])
            gene_as <- rbind(gene_as,res)
        }
        
    } #if has sig. events
    
} #for all files
n_unique_genes <- length(summary(as.factor(gene_as$GeneSymbol),maxsum=50000))
message("We extracted a total of ",nrow(gene_as)," significant alternative splicing events (gene_as)")
message("This includes ", n_unique_genes, " total genes")

We extracted a total of 2848 significant alternative splicing events (gene_as)

This includes 1677 total genes



### 3 Data Structures for Figures

### 3.1 gene_as.tsv

This file contains (description)
Here is a typical line
<pre>
        GeneJunction    ASE     ASE_IDX Tissue  GeneSymbol      chr
1       MDM4-3553       SE      3553    adipovisceral_omentum   RNPEP   chr1
2       WDR17-8668      SE      8668    adipovisceral_omentum   ANKMY1  chr2
3       IL17RC-5032     SE      5032    adipovisceral_omentum   SNCAIP  chr5
4       DDX3X-5712      A3SS    5712    adrenal_gland   DDX3X   chrX
</pre>
There are 2848 significant events in the file.

In [34]:
glimpse(gene_as)
gene_as$Tissue <- factor(gene_as$Tissue)
length(levels(gene_as$Tissue))
table(is.na(gene_as$Display))
table(gene_as$Display)
colnames(gene_as)
write.table(gene_as, "../data/gene_as.tsv", quote=FALSE, sep="\t")
head(gene_as)
tissue_reduction <- factor(tissue_reduction$display_name)

Observations: 2,848
Variables: 9
$ GeneJunction <fct> MDM4-3553, WDR17-8668, IL17RC-5032, DDX3X-5712, SCO1-845…
$ ASE          <fct> A3SS, A3SS, A3SS, A3SS, A3SS, A3SS, A3SS, A3SS, A3SS, A3…
$ ASE_IDX      <int> 3553, 8668, 5032, 5712, 8452, 1493, 5712, 5710, 4891, 45…
$ Tissue       <fct> adipose_visceral_omentum, adipose_visceral_omentum, adip…
$ counts       <int> 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 3, 3, 3,…
$ Display      <fct> Adipose (v), Adipose (v), Adipose (v), Adrenal gland, Ad…
$ GeneSymbol   <fct> MDM4, WDR17, IL17RC, DDX3X, SCO1, MYO7A, DDX3X, DDX3X, H…
$ GeneID       <fct> ENSG00000198625.13, ENSG00000150627.15, ENSG00000163702.…
$ chr          <fct> chr1, chr4, chr3, chrX, chr17, chr11, chrX, chrX, chr1, …


[1] 38


FALSE 
 2848 


         Adipose (sc)           Adipose (v)         Adrenal gland 
                   26                    23                    20 
                Aorta      Atrial appendage                Breast 
                   92                    12                  1788 
              Caudate Cerebellar hemisphere            Cerebellum 
                    3                     1                    22 
      Coronary artery                Cortex       EBV-lymphocytes 
                    4                     6                     7 
      Esophagus (gej)         Esophagus (m)        Esophagus (mu) 
                   14                    17                   241 
          Fibroblasts        Frontal cortex           Hippocampus 
                   37                     0                     9 
         Hypothalamus        Left ventricle                 Liver 
                    2                     7                     6 
                 Lung     Nucleus accumbens              Panc

[1] "GeneJunction" "ASE"          "ASE_IDX"      "Tissue"       "counts"      
[6] "Display"      "GeneSymbol"   "GeneID"       "chr"

,GeneJunction,ASE,ASE_IDX,Tissue,counts,Display,GeneSymbol,GeneID,chr
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>
1,MDM4-3553,A3SS,3553,adipose_visceral_omentum,3,Adipose (v),MDM4,ENSG00000198625.13,chr1
2,WDR17-8668,A3SS,8668,adipose_visceral_omentum,3,Adipose (v),WDR17,ENSG00000150627.15,chr4
3,IL17RC-5032,A3SS,5032,adipose_visceral_omentum,3,Adipose (v),IL17RC,ENSG00000163702.20,chr3
4,DDX3X-5712,A3SS,5712,adrenal_gland,3,Adrenal gland,DDX3X,ENSG00000215301.10,chrX
5,SCO1-8452,A3SS,8452,adrenal_gland,3,Adrenal gland,SCO1,ENSG00000133028.12,chr17
6,MYO7A-1493,A3SS,1493,adrenal_gland,3,Adrenal gland,MYO7A,ENSG00000137474.21,chr11


ERROR: Error in tissue_reduction$display_name: $ operator is invalid for atomic vectors


In [39]:
head(gene_as[gene_as$chr=="chrX",])
x_as_events <- gene_as[gene_as$chr=="chrX",]
message("There were ",nrow(gene_as)," total significant alternative splicing events (gene_as)")
message("There were ",nrow(x_as_events)," total significant alternative splicing events on the X chromosome (gene_as)")
message("i.e., ", (100*nrow(x_as_events)/nrow(gene_as)), "% of all significant AS events were on the X chromosome")

,GeneJunction,ASE,ASE_IDX,Tissue,counts,Display,GeneSymbol,GeneID,chr
,<fct>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>
4,DDX3X-5712,A3SS,5712,adrenal_gland,3,Adrenal gland,DDX3X,ENSG00000215301.10,chrX
7,DDX3X-5712,A3SS,5712,artery_aorta,8,Aorta,DDX3X,ENSG00000215301.10,chrX
8,DDX3X-5710,A3SS,5710,artery_aorta,8,Aorta,DDX3X,ENSG00000215301.10,chrX
39,DDX3X-5706,A3SS,5706,breast_mammary_tissue,146,Breast,DDX3X,ENSG00000215301.10,chrX
111,SPIN3-6181,A3SS,6181,breast_mammary_tissue,146,Breast,SPIN3,ENSG00000204271.13,chrX
146,IGSF1-8876,A3SS,8876,breast_mammary_tissue,146,Breast,IGSF1,ENSG00000147255.19,chrX


There were 2848 total significant alternative splicing events (gene_as)

There were 254 total significant alternative splicing events on the X chromosome (gene_as)

i.e., 8.9185393258427% of all significant AS events were on the X chromosome



### 3.2 Tissue specific data frame

In [40]:
data <- data.frame(Tissue=gene_as$Tissue, ASE=gene_as$ASE, Counts=gene_as$counts)

numberOfUniqueTissues <- length(summary(as.factor(data$Tissue),maxsum=500))
numberOfASEmechanisms <- length(summary(as.factor(data$ASE),maxsum=500))

message("data now has ",numberOfUniqueTissues, " tissues and ", numberOfASEmechanisms, " ASE categories")
message("ASE:")
summary(as.factor(data$ASE),maxsum=500)

data now has 38 tissues and 5 ASE categories

ASE:



A3SS A5SS  MXE   RI   SE 
 232  207  121  192 2096

### 3.3 Count splicing event by chromosome

Count the number of significant alternative splicing events per chromosome and write to the file **Total_AS_by_chr.tsv**.

In [41]:
res2 <- gene_as          %>% 
       group_by(chr)    %>% 
       count(chr)       %>% 
       arrange(desc(n)) %>% 
       as.data.frame()
res2$chr <- factor(res2$chr, levels = res2$chr)
length(res2$chr)
res2
glimpse(res2)
write.table(res2, file= "../data/Total_AS_by_chr.tsv", sep="\t", quote = FALSE, row.names=F)

[1] 23

chr,n
<fct>,<int>
chrX,254
chr1,236
chr11,184
chr2,180
chr12,175
chr19,167
chr17,165
chr3,157
chr16,148


Observations: 23
Variables: 2
$ chr <fct> chrX, chr1, chr11, chr2, chr12, chr19, chr17, chr3, chr16, chr4, …
$ n   <int> 254, 236, 184, 180, 175, 167, 165, 157, 148, 133, 118, 115, 107, …


### 3.4 Count most frequent spliced genes 

In [42]:
res3 <- gene_as %>% 
       group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
res3$GeneSymbol <- factor(res3$GeneSymbol, levels = res3$GeneSymbol)
length(res3$GeneSymbol)
head(res3)
write.table(res3, file = "../data/Total_AS_by_geneSymbol.tsv", sep = "\t", quote=FALSE, row.names = F)

[1] 1677

,GeneSymbol,n
,<fct>,<int>
1,DDX3X,85
2,KDM5C,40
3,ZFX,21
4,SORBS2,18
5,CD44,16
6,CACNB4,12


### 3.5 Count most frequent splicing by tissue

In [43]:
res4 <- gene_as %>% 
       group_by(Display) %>% 
       count(Display) %>% 
       arrange(desc(n)) %>% 
       as.data.frame()
res4$Display <- factor(res4$Display, levels = res4$Display)
length(res4$Display)
res4
write.table(res4, file = "../data/Total_AS_by_tissue.tsv", sep = "\t", row.names = F)

[1] 38

Display,n
<fct>,<int>
Breast,1788
Nucleus accumbens,271
Esophagus (mu),241
Aorta,92
Thyroid,52
Spleen,40
Fibroblasts,37
Skeletal muscle,31
Adipose (sc),26


###  3.6 Significant Count by splicing type 
We define **significant** to be FC > 1.5 and pVal < 0.05

Our starting values were the significant events, all meeting the criteria FC > 1.5 and pVal < 0.05


In [44]:
res5 <- gene_as %>% group_by(ASE) %>% count(ASE) %>% arrange(desc(n)) %>% as.data.frame()
res5$ASE <- factor(res5$ASE, levels = res5$ASE)
head(res5)
write.table(res5, file= "../data/Total_AS_by_splicingtype.tsv")

,ASE,n
,<fct>,<int>
1,SE,2096
2,A3SS,232
3,A5SS,207
4,RI,192
5,MXE,121


###  3.7 Significant Count by splicing type (significant == FC > 1.5 and pVal < 0.05)

In [45]:
A3SS_keep <- as.character(gene_as$ASE) %in% "A3SS"
table(A3SS_keep)
A3SS.gene_as <- data.frame(gene_as[A3SS_keep == TRUE,])

A5SS_keep <- as.character(gene_as$ASE) %in% "A5SS"
table(A5SS_keep)
A5SS.gene_as <- data.frame(gene_as[A5SS_keep == TRUE,])

MXE_keep  <- as.character(gene_as$ASE) %in% "MXE"
table(MXE_keep)
MXE.gene_as <- data.frame(gene_as[MXE_keep == TRUE,])

SE_keep   <- as.character(gene_as$ASE) %in% "SE"
table(SE_keep)
SE.gene_as <- data.frame(gene_as[SE_keep == TRUE,])

RI_keep   <- as.character(gene_as$ASE) %in% "RI"
table(RI_keep)
RI.gene_as <- data.frame(gene_as[RI_keep == TRUE,])

dim(A3SS.gene_as)
dim(A5SS.gene_as)
dim(MXE.gene_as)
dim(SE.gene_as)
dim(RI.gene_as)


A3SS_keep
FALSE  TRUE 
 2616   232 

A5SS_keep
FALSE  TRUE 
 2641   207 

MXE_keep
FALSE  TRUE 
 2727   121 

SE_keep
FALSE  TRUE 
  752  2096 

RI_keep
FALSE  TRUE 
 2656   192 

[1] 232   9

[1] 207   9

[1] 121   9

[1] 2096    9

[1] 192   9

### 3.8 Siginficant spliced by Gene for each splicing factor

In [46]:
A3SS.res <- A3SS.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
A3SS.res$GeneSymbol <- factor(A3SS.res$GeneSymbol, levels = A3SS.res$GeneSymbol)
message("Significant spliced genes for A3SS\n",
        paste(length(A3SS.res$GeneSymbol)), collapse=" ")
head(A3SS.res)

A5SS.res <- A5SS.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
A5SS.res$GeneSymbol <- factor(A5SS.res$GeneSymbol, levels = A5SS.res$GeneSymbol)
message("Significant spliced genes for A5SS\n",
        paste(length(A5SS.res$GeneSymbol)), collapse=" ")
head(A5SS.res)

MXE.res <- MXE.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
MXE.res$GeneSymbol <- factor(MXE.res$GeneSymbol, levels = MXE.res$GeneSymbol)
message("Significant spliced genes for MXE\n",
        paste(length(MXE.res$GeneSymbol)), collapse=" ")
head(MXE.res)

RI.res <- RI.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
RI.res$GeneSymbol <- factor(RI.res$GeneSymbol, levels = RI.res$GeneSymbol)
message("Significant spliced genes for RI\n",
        paste(length(RI.res$GeneSymbol)), collapse=" ")
head(RI.res)

SE.res <- SE.gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
SE.res$GeneSymbol <- factor(SE.res$GeneSymbol, levels = SE.res$GeneSymbol)
message("Significant spliced genes for SE\n",
        paste(length(SE.res$GeneSymbol)), collapse=" ")
head(SE.res)

Significant spliced genes for A3SS
187 



,GeneSymbol,n
,<fct>,<int>
1,DDX3X,13
2,HAND2-AS1,7
3,WDR17,3
4,ADGRG1,3
5,PPM1J,3
6,FAIM2,2


Significant spliced genes for A5SS
170 



,GeneSymbol,n
,<fct>,<int>
1,DDX3X,23
2,WDR31,4
3,ITGB7,3
4,DEPDC5,2
5,SGCE,2
6,CD38,2


Significant spliced genes for MXE
88 



,GeneSymbol,n
,<fct>,<int>
1,DDX3X,7
2,SORBS2,5
3,ACSL6,4
4,SGCE,3
5,NRG2,3
6,FAM49B,3


Significant spliced genes for RI
160 



,GeneSymbol,n
,<fct>,<int>
1,DDX3X,13
2,HAND2-AS1,3
3,SGCE,3
4,SPG7,3
5,TMEM79,3
6,KDM5B,2


Significant spliced genes for SE
1335 



,GeneSymbol,n
,<fct>,<int>
1,KDM5C,40
2,DDX3X,29
3,ZFX,21
4,CD44,14
5,SORBS2,12
6,CACNB4,11


### 3.9 Count most frequent spliced genes

In [47]:
res <- gene_as %>% group_by(GeneSymbol) %>% count(GeneSymbol) %>% arrange(desc(n)) %>% as.data.frame()
res$GeneSymbol <- factor(res$GeneSymbol, levels = res$GeneSymbol)
length(res$GeneSymbol)
res2 <- data %>% group_by(Tissue) %>% 
    summarise(Total = sum(Counts)) %>%
    arrange(desc(Total)) %>%
    as.data.frame()

#Add number of tissues
nTissues <- rep(NA, length(res))
for (i in 1:nrow(res)) {
  df_gene <- gene_as %>% filter(GeneSymbol == res$GeneSymbol[i])
  nTissues[i] <- length(unique(df_gene$Tissue))
}
res$Tissues <- nTissues
head(res)
write.table(res, file = "../data/genesWithCommonAS.tsv", sep = "\t", quote = F, row.names = F)

[1] 1677

,GeneSymbol,n,Tissues
,<fct>,<int>,<int>
1,DDX3X,85,19
2,KDM5C,40,29
3,ZFX,21,13
4,SORBS2,18,2
5,CD44,16,1
6,CACNB4,12,1


### 3.10 Count most frequent spliced chromosomes
To get an indication of which chromosome has the most frequent slicing event (regardless of type)
We create an index based upon the number of exons per chromosome.

get the annotation file, at this writing, gencode.v30.annotation.gtf
The information as to the number of exons within the chromosome may be found there

In [48]:
if (!("gencode.v30.annotation.gtf.gz" %in% list.files("../data/"))) {
    message("downloading gencode v30 annotation\n")
    system("wget -O ../data/gencode.v30.annotation.gtf.gz ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_30/gencode.v30.annotation.gtf.gz")
    message("Done!\n")
    message("Unzipping compressed file gencode.v30.annotation.gtf.gz..")
    system("gunzip ../data/gencode.v30.annotation.gtf.gz", intern = TRUE)
    message("Done! gencode.v30.annotation.gtf can be found in ../data/")
}
gencode <- import("../data/gencode.v30.annotation.gtf")

downloading gencode v30 annotation


Done!


Unzipping compressed file gencode.v30.annotation.gtf.gz..

Done! gencode.v30.annotation.gtf can be found in ../data/



In [49]:
exons <- gencode[ gencode$type == "exon", ]
exons <- as.data.frame(exons)

#Obtain chromosomes we have splicing information for (recall we did not use chr Y in our analysis)
all_chr <- as.character(unique(gene_as$chr))
chr_counts <- rep(0, length(all_chr))


for (i in 1:length(all_chr)) {
  chr_counts[i] <- nrow(exons[exons$seqnames == all_chr[i], ])
}

exon_counts <- data.frame(chr = all_chr, counts = chr_counts)

# Count most frequent spliced chromosomes
res <- gene_as %>% group_by(chr) %>% count(chr) %>% arrange(desc(n)) %>% as.data.frame()
res$chr <- factor(res$chr, levels = res$chr)

idx <- match(res$chr, exon_counts$chr)

res$ExonCounts <- exon_counts$counts[idx]

res$Index <- (res$n / res$ExonCounts) * 1000

res_sorted <- res %>% arrange(desc(Index))
res_sorted$chr <- factor(res_sorted$chr, levels = res_sorted$chr)
glimpse(res_sorted)

Observations: 23
Variables: 4
$ chr        <fct> chrX, chr22, chr4, chr15, chr11, chr16, chr12, chr10, chr1…
$ n          <int> 254, 98, 133, 115, 184, 148, 175, 106, 167, 101, 49, 165, …
$ ExonCounts <dbl> 40029, 28655, 50420, 47343, 75976, 61199, 73881, 47124, 74…
$ Index      <dbl> 6.345400, 3.419997, 2.637842, 2.429081, 2.421817, 2.418340…


In [50]:
write.table(data,       file = "../data/Significant_AS_events.tsv", sep = "\t", row.names = F, quote = F)
write.table(res_sorted, file = "../data/SplicingIndex_chr.tsv", sep = "\t", quote = F, row.names = F)

# Count gene expression analysis results
The files called (tissue)\_DGE\_refined.csv contain lists of genes found to have statistically significant differential expression.
The mapping files contain the ENSG id to gene symbol maps.

In [56]:
significant_results_dir = "../data/"
pattern = "_DGE_refined.csv"
files <- list.files(path = significant_results_dir, pattern = pattern)
map_pattern <- "_DGE_ensg_map.csv"
map_files <- list.files(path = significant_results_dir, pattern = map_pattern)
message("We got ", length(files), " files with significant DGEs and ", length(map_files), " mapping files")

We got 39 files with significant DGEs and 39 mapping files



In [58]:
gene_dge = data.frame()
Tissue <- rep("NA", length(files))
Display <- rep("NA", length(files))
logFC <- rep(NA, length(files))
AveExpr <- rep("NA", length(files))
t <- rep("NA", length(files))
PValue <- rep("NA", length(files))
AdjPVal <- rep("NA", length(files))
GeneID <- rep("NA", length(files))
chr <- rep("NA", length(files))

for (i in 1:length(files)) {
    lines <- read.csv(paste0(significant_results_dir, files[i]),header = TRUE)
    ensid     <- as.vector(as.character(rownames(lines)))
    tissue1 <- gsub("_DGE_refined.csv","", files[i], fixed = TRUE)
    tissue2 <- substring(tissue1,6)  # remove the first 5 letters of the string 
    Tissue[i] <- tissue2
    Display[i] <- tissue_reduction[tissue_reduction$SMTSD == tissue2, "display_name"]
    counts[i] <- dim(lines)[1]    
            
}


ERROR: Error in library(hash): there is no package called ‘hash’


### Appendix - Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **'artefacts'**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### Appendix 1. Checksums with the sha256 algorithm

In [25]:
rm (notebookid)
notebookid   = "countGenesAndEvents"
notebookid

message("Generating sha256 checksums of the file `../data/Total_AS_by_tissue.tsv` directory .. ")
system(paste0("cd ../data && find . -name SplicingIndex_chr.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Significant_events.tsv` directory .. ")
system(paste0("cd ../data && find . -name SplicingIndex_chr.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

message("Generating sha256 checksums of the file `../data/Significant_events.tsv` directory .. ")
system(paste0("cd ../data && find . -name SplicingIndex_chr.tsv -exec sha256sum {} \\;  >  ../metadata/", notebookid, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")


paste0("../metadata/", notebookid, "_sha256sums.txt")

data.table::fread(paste0("../metadata/", notebookid, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

Warning message in rm(notebookid):
“object 'notebookid' not found”


[1] "countGenesAndEvents"

Generating sha256 checksums of the file `../data/Total_AS_by_tissue.tsv` directory .. 



character(0)

Done!


Generating sha256 checksums of the file `../data/Significant_events.tsv` directory .. 



...

Done!


Generating sha256 checksums of the file `../data/Significant_events.tsv` directory .. 



character(0)

Done!




[1] "../metadata/countGenesAndEvents_sha256sums.txt"

sha256sum,file
<chr>,<chr>
3d4cd316ca1e042d5fb5d6bea4bb73e69d2c024956caa09119c0d80dde5d8f65,./SplicingIndex_chr.tsv


### Appendix 2. Libraries metadata

In [26]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", notebookid, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", notebookid ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]

Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..

Done!


Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..

Done!




 setting  value                       
 version  R version 3.6.2 (2019-12-12)
 os       Ubuntu 18.04.3 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language en_US.UTF-8                 
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2020-06-16                  

,package,ondiskversion,loadedversion,path,loadedpath,attached,is_base,date,source,md5ok,library
,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,<lgl>,<fct>
Biobase,Biobase,2.46.0,2.46.0,/opt/conda/lib/R/library/Biobase,/opt/conda/lib/R/library/Biobase,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
BiocGenerics,BiocGenerics,0.32.0,0.32.0,/opt/conda/lib/R/library/BiocGenerics,/opt/conda/lib/R/library/BiocGenerics,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
dplyr,dplyr,0.8.4,0.8.4,/opt/conda/lib/R/library/dplyr,/opt/conda/lib/R/library/dplyr,TRUE,FALSE,2020-01-31,CRAN (R 3.6.2),NA,/opt/conda/lib/R/library
edgeR,edgeR,3.28.0,3.28.0,/opt/conda/lib/R/library/edgeR,/opt/conda/lib/R/library/edgeR,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
GenomeInfoDb,GenomeInfoDb,1.22.0,1.22.0,/opt/conda/lib/R/library/GenomeInfoDb,/opt/conda/lib/R/library/GenomeInfoDb,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
GenomicRanges,GenomicRanges,1.38.0,1.38.0,/opt/conda/lib/R/library/GenomicRanges,/opt/conda/lib/R/library/GenomicRanges,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
ggplot2,ggplot2,3.2.1,3.2.1,/opt/conda/lib/R/library/ggplot2,/opt/conda/lib/R/library/ggplot2,TRUE,FALSE,2019-08-10,CRAN (R 3.6.1),NA,/opt/conda/lib/R/library
IRanges,IRanges,2.20.0,2.20.0,/opt/conda/lib/R/library/IRanges,/opt/conda/lib/R/library/IRanges,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
limma,limma,3.42.0,3.42.0,/opt/conda/lib/R/library/limma,/opt/conda/lib/R/library/limma,TRUE,FALSE,2019-10-29,Bioconductor,NA,/opt/conda/lib/R/library
